In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#https://analyticsindiamag.com/beginners-guide-to-truncated-svd-for-dimensionality-reduction/
#https://chrisalbon.com/code/machine_learning/feature_engineering/select_best_number_of_components_in_tsvd/

In [3]:
from src.data import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
import joblib

In [4]:
params = preprocessing.dict_params
INPUT_FILE_PATH = './../data/raw'
OUTPUT_FILE_PATH = './../data/interim'
INDEX_KEY = [0] # the 1st column is the primary key

## Load preprocessed data

In [5]:
interim_ingred = pd.read_csv(os.path.join(OUTPUT_FILE_PATH,preprocessing.get_param('ingredients', 'file_name')))
print(interim_ingred.shape)
interim_ingred.head()

(2479, 226)


,Code_AGB,Code_CIQUAL,Groupe_aliment,Sous-groupe_aliment,Nom_Produit_Francais,LCI_Name,Saisonnalite,Transport_par_avion_,Livraison,Materiau_emballage,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
0,13712,13712,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, égoutté","Apricot, canned in light syrup, drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13713,13713,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, non égoutté","Apricot, canned in light syrup, not drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13714,13714,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, égoutté","Apricot, in syrup, canned, drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13715,13715,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, non égoutté","Apricot, in syrup, canned, not drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13000,13000,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot, dénoyauté, cru","Apricot, pitted, raw",mix de consommation FR,0,Ambiant (moyenne),LPDE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
for i in interim_ingred:
    print(i)

Code_AGB
Code_CIQUAL
Groupe_aliment
Sous-groupe_aliment
Nom_Produit_Francais
LCI_Name
Saisonnalite
Transport_par_avion_
Livraison
Materiau_emballage
Preparation
DQR_Note_qualite_la_donnee_
Score_unique_EF_
Abats de bœuf
Abats de porc
Abats de poulet
Abats de veau
Abricot
Ail
Amande
Amidon de maïs
Ananas
Anchois
Anchois commun
Asperges
Aubergine
Autres étapes
Avocat
Avoine
Baie
Basilic
Beurre
Bière
Blanc d'oeuf
Blé dur
Boisson à base d'avoine
Boisson à base de soja
Bouillon
Brocoli
Bœuf haché
Cacahuète
Cacao
Café
Calamar
Camembert
Cannelle 
Carotte
Cassonade
Cerises
Champignon
Chicorée
Chicorée rouge
Chocolat au lait
Chocolat noir
Chou blanc
Chou vert
Choucroute
Choufleur
Châtaigne
Ciboulette ou oignon de printemps
Clous de girofle
Comté
Concentré de tomates 30° Brix
Concombre
Congre
Coquille Saint-Jacques
Coriandre
Courgette
Cresson de jardin
Crevette
Crème
Céleri
Céleri-rave
Eau de Vie
Emmental
Endive
Endive d'hiver
Epinard
Escargot
Estragon
Farine de blé
Farine de riz
Farine de sarra

In [18]:
interim_ingred.columns[13]

'Abats de bœuf'

In [19]:
#Define feature matrix
X=interim_ingred.iloc[:,13:]
print(X.shape)
X.head()

(2479, 213)


,Abats de bœuf,Abats de porc,Abats de poulet,Abats de veau,Abricot,Ail,Amande,Amidon de maïs,Ananas,Anchois,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#Clean feature matrix
#Get rid of singular ingredients (products that are not composed of multiple ingredients but are a single ingredient by themselves. E.g. broccoli)
single_ing=X.sum(axis=1)!=0
print('Feature matrix shape before taking out single ingredients:', X.shape)
X_clean=X[single_ing]
print('Feature matrix shape after taking out single ingredients:', X_clean.shape)


Feature matrix shape before taking out single ingredients: (2479, 213)
Feature matrix shape after taking out single ingredients: (1038, 213)


In [21]:
#Also clean target:
Y_clean=interim_ingred[single_ing]['Score_unique_EF_']
len(Y_clean)

1038

## Dimensionality reduction using Truncated SVD  

Since we have a sparse matrix, use truncated SVD to reduce dimensions of matrix for training of model

### Define TSVD

In [22]:
#First: Create and run a TSVD with  n_components equal one less than number of features
tsvd_model = TruncatedSVD(n_components=X_clean.shape[1]-1)
tsvd_model.fit(X_clean)
#X_tsvd=tsvd_model.fit(X_clean)


#List of explained variances
tsvd_explained_var_ratios = tsvd_model.explained_variance_ratio_
tsvd_explained_var_ratios

array([3.97523621e-02, 9.41758359e-02, 6.70510717e-02, 5.99316174e-02,
       5.16268680e-02, 4.64063359e-02, 3.72498530e-02, 3.42478253e-02,
       2.96292479e-02, 2.67849810e-02, 2.43002698e-02, 2.09182001e-02,
       1.92771899e-02, 1.83205937e-02, 1.74223276e-02, 1.53563099e-02,
       1.44284128e-02, 1.35623876e-02, 1.33138690e-02, 1.21475982e-02,
       1.19998391e-02, 1.10723798e-02, 1.05926233e-02, 1.01855873e-02,
       9.65919412e-03, 9.08031665e-03, 8.82980169e-03, 8.51948300e-03,
       8.40119773e-03, 8.17309250e-03, 7.98098824e-03, 7.66015292e-03,
       7.44576840e-03, 7.09255165e-03, 6.89034852e-03, 6.51785305e-03,
       5.95628991e-03, 5.81677823e-03, 5.57794022e-03, 5.23830072e-03,
       5.11238441e-03, 4.83688226e-03, 4.81571653e-03, 4.54742500e-03,
       4.45097522e-03, 4.17915195e-03, 4.06833861e-03, 4.02422046e-03,
       3.93156924e-03, 3.73828909e-03, 3.69250359e-03, 3.58372903e-03,
       3.49226408e-03, 3.33382021e-03, 3.23069392e-03, 3.21063863e-03,
      

### Define number of features explaining at least 80% of variance 

In [23]:
# Create a function to select n_components to reduce X_clean using TSVD. Those components should explain a chosen threshold percentage variance
def best_n_components(var_ratio, thresh_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= thresh_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [27]:
#Run function to get ideal number of components explaining 80% of variance of data
n_comp=best_n_components(tsvd_explained_var_ratios, 0.80)
n_comp

37

In [25]:
#Fit and transform X using TSVD
tsvd_model = TruncatedSVD(n_components=n_comp)
X_truncated = tsvd_model.fit_transform(X_clean)
print('Shape of truncated features matrix',X_truncated.shape)
X_truncated

Shape of truncated features matrix (1038, 37)


array([[ 1.54872333,  0.30387433, -0.42128241, ...,  0.02283316,
        -0.02789424, -0.07318835],
       [ 1.28378599,  0.19434964, -0.47672928, ...,  0.07751685,
         0.28631837, -0.06339439],
       [ 0.78366355,  0.41662218,  0.23413864, ...,  0.02240495,
        -0.090803  , -0.10344706],
       ...,
       [ 0.95100513,  0.29494501,  0.36481699, ...,  0.01210504,
        -0.09301387, -0.07269536],
       [ 1.04621071,  0.35087547,  0.3761776 , ...,  0.06212785,
        -0.07454749, -0.07753212],
       [ 1.24372238, -0.09522632,  0.49484023, ...,  0.0351766 ,
        -0.0610509 ,  0.01854169]])

In [26]:
#Visualize X_truncated
df_plot=pd.DataFrame(X_truncated,index=X_clean.index)
df_plot['Target']=Y_clean
df_plot['Groupe_aliment']=interim_ingred[single_ing]['Groupe_aliment']
df_plot.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,Target,Groupe_aliment
6,1.548723,0.303874,-0.421282,-0.026037,0.792932,-0.609456,-0.182417,-0.031637,-0.246680,0.091042,...,0.103268,0.152632,0.324416,-0.073300,-0.016836,0.022833,-0.027894,-0.073188,0.907318,"viandes, œufs, poissons"
30,1.283786,0.194350,-0.476729,0.984187,0.086225,-0.378690,0.808025,-0.050250,0.372315,-0.555809,...,0.026829,0.191401,-0.033205,0.057686,-0.336477,0.077517,0.286318,-0.063394,0.370846,entrées et plats composés
43,0.783664,0.416622,0.234139,0.077693,0.074010,0.431521,-0.106030,0.743700,0.020323,-0.437635,...,-0.069803,0.118113,0.210612,-0.179803,0.018552,0.022405,-0.090803,-0.103447,0.265689,"viandes, œufs, poissons"
44,0.783664,0.416622,0.234139,0.077693,0.074010,0.431521,-0.106030,0.743700,0.020323,-0.437635,...,-0.069803,0.118113,0.210612,-0.179803,0.018552,0.022405,-0.090803,-0.103447,0.258212,"viandes, œufs, poissons"
45,0.783664,0.416622,0.234139,0.077693,0.074010,0.431521,-0.106030,0.743700,0.020323,-0.437635,...,-0.069803,0.118113,0.210612,-0.179803,0.018552,0.022405,-0.090803,-0.103447,0.258212,"viandes, œufs, poissons"
